In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os
import sys
sys.path.append(os.path.join(os.getcwd(), 'baseCode', 'models'))
from classification import load_model, print_layers
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
# Configuración de parámetros
backbone = 'vgg16'  # Cambia por el backbone que desees
weights = 'imagenet'  # 'none' o 'imagenet'
num_classes = 2
num_epochs = 5
learning_rate = 0.001
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo seleccionado: {device}")

In [ ]:

metadata_path = os.path.join(os.getcwd(), 'E:/DataTIC/ISIC-images', 'E:/DataTIC/ISIC-images/metadata.csv')
images_dir = os.path.join(os.getcwd(), 'E:/DataTIC/ISIC-images', '/')  

df = pd.read_csv(metadata_path)
label_col = 'diagnosis_1' 
labels = df[label_col].fillna('Desconocido').unique()
label2idx = {label: idx for idx, label in enumerate(labels)}

class MetadataImageDataset(Dataset):
    def __init__(self, dataframe, images_dir, label_col, transform=None):
        self.dataframe = dataframe
        self.images_dir = images_dir
        self.label_col = label_col
        self.transform = transform
        self.label2idx = label2idx
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_name = row['isic_id'] + '.jpg'
        img_path = os.path.join(self.images_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.label2idx[row[self.label_col] if pd.notnull(row[self.label_col]) else 'Desconocido']
        if self.transform:
            image = self.transform(image)
        return image, label

train_dataset = MetadataImageDataset(df, images_dir, label_col, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(f"Número de clases detectadas: {len(label2idx)}")

In [ ]:
model = load_model(backbone, weights, num_classes)
model = model.to(device)
print_layers(model)
print(f"Modelo {backbone} cargado con {num_classes} clases y pesos: {weights}")

In [ ]:

data_dir = os.path.join(os.getcwd(), 'database', 'train')  
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

try:
    train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    print(f"Número de clases detectadas: {len(train_dataset.classes)}")
except Exception as e:
    print("No se encontró un dataset válido en la ruta especificada. Se usará un dataset sintético para ejemplo.")
    from torch.utils.data import Dataset
    class DummyDataset(Dataset):
        def __init__(self, length=100):
            self.length = length
        def __len__(self):
            return self.length
        def __getitem__(self, idx):
            x = torch.randn(3, 224, 224)
            y = torch.randint(0, num_classes, (1,)).item()
            return x, y
    train_dataset = DummyDataset()
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Definir función de entrenamiento
def train_model(model, train_loader, criterion, optimizer, device, num_epochs):
    history = {'loss': [], 'acc': []}
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        epoch_loss = running_loss / total
        epoch_acc = correct / total
        history['loss'].append(epoch_loss)
        history['acc'].append(epoch_acc)
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f}")
    return history

In [ ]:
# Entrenar el modelo
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
history = train_model(model, train_loader, criterion, optimizer, device, num_epochs)

# Guardar el modelo entrenado
model_path = f"modelo_{backbone}.pth"
torch.save(model.state_dict(), model_path)
print(f"Modelo guardado en {model_path}")

In [ ]:
# Visualizar resultados del entrenamiento
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history['loss'], marker='o')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(history['acc'], marker='o', color='green')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()